# 第二章 OpenAI 函数调用 OpenAI Function Calling

本章主要介绍 OpenAI 的 API 提供的新功能，在最近几个月，OpenAI 对一些新的模型进行了微调，分别是`gpt-3.5-turbo-0613`和`gpt-4-0613`。

假如我们有一个工具函数，在一些特定情况下需要调用，经过微调后，这些模型能够传入新的参数，可以通过这个新的参数来自动判断是否调用工具函数，如果判断需要调用工具函数，会返回这个工具函数和对应的输入参数。


- [一、OpenAI函数新参数](#一openai函数新参数)
  - [1.1 简单例子：得到当前天气](#11-简单例子得到当前天气)  
  - [1.2 新参数：functions](#12-新参数functions) 
  - [1.3 相关提示调用结果](#13-相关提示调用结果)    
  - [1.4 无关提示调用结果](#14-无关提示调用结果)
- [二、Function Call参数模式](#二function-call参数模式)
  - [2.1 自动判断是否调用](#21-自动判断是否调用)    
  - [2.2 强制不调用](#22-强制不调用)    
  - [2.3 强制调用](#23-强制调用)  
- [三、函数调用以及执行函数](#三函数调用以及执行函数)   
- [四、英文版提示](#四英文版提示)

## 一、OpenAI函数新参数

首先我们直接定义使用`OPENAI_API_KEY`，方便后续去调用OpenAI的API接口，并利用其函数。

In [1]:
import os
import openai

os.environ['OPENAI_API_KEY'] = "your_api_key"
openai.api_key = os.environ['OPENAI_API_KEY']

### 1.1 简单例子：得到当前天气

首先我们使用 OpenAI 的使用的第一个例子，定义了一个`get_current_weather`的函数，正常来说，获取当前天气是语言模型本身不能完全做到的事情。因此，我们希望能够将语言模型和这样的函数结合起来，以当前的信息来增强它。

在这个函数，我们固定了返回的值，比如温度固定为 22 摄氏度，但在实际应用中，这可能涉及到调用天气   API 或一些外部知识源。

In [2]:
import json

# 定义一个函数，用于获取给定位置的当前天气
def get_current_weather(location, unit="摄氏度"):
    """获取指定位置的当前天气"""
    # 模拟返回相同的天气情况的示例函数
    # 在实际应用环境中，这可以是天气API
    # 创建一个天气信息的字典
    weather_info = {
        "location": location,  # 天气的位置
        "temperature": "22",  # 温度
        "unit": unit,  # 温度单位，默认为摄氏度
        "forecast": ["晴", "多云"],  # 天气预报
    }
    # 将天气信息转换为JSON格式的字符串并返回
    return json.dumps(weather_info)

### 1.2 新参数：functions

那我们如何将这样的函数传给语言模型呢？OpenAI引入了一个名为`functions`的新参数，通过该参数，我们可以传递一个函数定义列表。由于我们只有一个函数，所以列表中只有一个元素。这是一个JSON对象，具有几个不同的参数。

- name：函数的名称
- description：函数的描述
- parameters：参数对象，里面有一些属性设置
  - type：类型
  - properties：本身是一个对象，传入的是对应的参数的描述。在例子中我们可以看到有两个元素，`location`和`unit`。每个这些元素都有一个类型，即字符串，然后是一个描述。`unit`是一个外部参数的设置，比如这里我们希望它可以是摄氏度或华氏度，所以我们可以在这里定义他的类型和以及枚举参数值。
  - required：必填的参数，比如这里我们需要的参数就是`location`。

在函数定义中，`description`以及在`properties`中的参数非常重要，因为这些将直接传递给语言模型，语言模型将使用这些描述来判断是否使用此函数。

In [3]:
# 定义一个函数
functions = [
    {
        "name": "get_current_weather",
        "description": "获取指定位置的当前天气情况",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "城市和省份，例如：北京，北京市",
                },
                "unit": {
                    "type": "string", 
                    "enum": ["摄氏度", "华氏度"]
                },
            },
            "required": ["location"],
        },
    }
]

### 1.3 相关提示调用结果

接下来我们就可以定义一个有关于天气的问题，如”北京的天气怎么样？“，然后使用 OpenAI 的函数进行调用对话的 API。首先我们得选取较新的模型，如`gpt-3.5-turbo-0613`，然后我们再将上述定义的        function 函数传入，查看最后的响应结果。

从结果上来看，返回的消息的角色是助手，内容为空，而是有一个函数调用参数`function_call`，其中包含两个对象，`name`和`arguments`。`Name`是`get_current_weather`，这与我们传递的函数的名称相同，然后`arguments`是这个 JSON 格式的字典，里面是我们需要的参数。

In [4]:
import openai
from openai import OpenAI
# 定义输入消息

# ! 注意：openai>=0.28 后更新了openai.ChatCompletion的用法，并且获取部分输出信息的方式也做出了调整
# 新的调用方式如下：
client = OpenAI(api_key="your_api_key")

# 调用OpenAI的ChatCompletion API获取响应
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages = [
    {
        "role": "user",
        "content": "北京的天气怎么样?使用摄氏度作为单位。"
    }],
    # messages=messages,
    functions=functions # 传入function参数
)

print(response)

ChatCompletion(id='chatcmpl-9bOYiPIa2h0UwGCQs2Lm07u0RURgP', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "北京，北京市",\n  "unit": "摄氏度"\n}', name='get_current_weather'), tool_calls=None))], created=1718699568, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=30, prompt_tokens=105, total_tokens=135))


我们也可以看到响应的参数，比如在这里的两个参数是`location`和`unit`。

In [5]:
# 打印传入参数
# ！ 注意：该方式是openai>=0.28后更新的部分，后续相关代码也需要作出同样的调整
print(response.choices[0].message.function_call.arguments)

{
  "location": "北京，北京市",
  "unit": "摄氏度"
}


如果我们仔细看响应的话，我们会发现内容现在是空的，`function_call`是一个字典，`function_call`中的`arguments`参数本身也是一个 JSON 字典。因此我们可以使用`json.loads`将其加载到 Python 字典中。它返回的参数可以直接传递给我们上述定义的`get_current_weather`函数。

我们会发现，使用 OpenAI 进行函数调用并不直接调用工具函数，我们还是需要去调用工具函数，它只是告诉我们要调用哪个函数，即名称，以及该函数的参数应该是什么。以及由于他并没有去执行函数，所以说，如果我们在使用`json.loads`解码的时候遇到了一些问题，那实际上是模型的问题，所以这一部分可以考虑在写工具函数的时候做一些保护措施，后续也会讨论这一个点。

In [6]:
# 获取response里面的message信息
response_message = response.choices[0].message

print(response_message)

ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "北京，北京市",\n  "unit": "摄氏度"\n}', name='get_current_weather'), tool_calls=None)


In [7]:
# 打印参数
print(response.choices[0].message.function_call.arguments)

{
  "location": "北京，北京市",
  "unit": "摄氏度"
}


In [8]:
# 将JSON格式的字符串转换为Python对象
args = json.loads(response_message.function_call.arguments)

# 调用get_current_weather函数并传入参数args
get_current_weather(args)

'{"location": {"location": "\\u5317\\u4eac\\uff0c\\u5317\\u4eac\\u5e02", "unit": "\\u6444\\u6c0f\\u5ea6"}, "temperature": "22", "unit": "\\u6444\\u6c0f\\u5ea6", "forecast": ["\\u6674", "\\u591a\\u4e91"]}'

### 1.4 无关提示调用结果

接下来，我们也探讨一下，如果问的问题与函数无关会产生什么样的，也就是与天气无关，会返回什么样的信息呢？从结果上来看，我们可以返回的内容正常并且没有`function_call`参数，也就是在语言模型中判断不使用工具函数。

In [9]:
# 与天气无关提示调用
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]


# 调用OpenAI的ChatCompletion API获取响应
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions # 传入function参数
)

print(response)

ChatCompletion(id='chatcmpl-9bOYjuL8cQTFoUWQqTGHyTdIeyzGd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='您好！有什么我可以帮助您的吗？', role='assistant', function_call=None, tool_calls=None))], created=1718699569, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=19, prompt_tokens=88, total_tokens=107))


由于编码问题，我们可以单独打印对应的返回的文本信息

In [10]:
print(response.choices[0].message.content)

您好！有什么我可以帮助您的吗？


## 二、Function Call参数模式

`function_call`参数一共有3种模式，我们可以传递其他参数`function_call`以强制模型使用或不使用函数。

1. 默认情况下，它设置为`auto`，也就是模型自行选择。
2. 在第二种模式中，我们可以强制它调用一个函数。如果我们希望始终返回函数
3. 另一种模式是`none`。这会强制语言模型不使用提供的任何函数。

### 2.1 自动判断是否调用

`auto`模式就是大模型自行选择是否要返回参数，这一部分也是默认的，上述所有的方式都是`auto`的模式

### 2.2 强制不调用

`none`模式对于这个例子来说并不重要，因为内容`你好`不需要函数调用，所以我们看到它没有被使用。

In [11]:
# 无关提示强制不调用
messages = [
    {
        "role": "user",
        "content": "你好",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none", # 传入参数强制不调用
)
print(response)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-9bOYkrVM2fkhnvg5R0cGcKGPHlKIa', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好！有什么我可以帮助您的吗？', role='assistant', function_call=None, tool_calls=None))], created=1718699570, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=88, total_tokens=106))
你好！有什么我可以帮助您的吗？


那如果我们在需要使用工具的函数时候强制不调用（也就是使用第二种模式）会出现什么结果呢？从结果上来看，他依然有正常的`role`和`content`，但是因为我们强制不调用函数，所以他在试图返回正确的参数，但是并不正确。

In [12]:
# 相关提示强制不调用
messages = [
    {
        "role": "user",
        "content": "北京天气怎么样?",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none", # 传入参数强制不调用
)
print(response)
print(response.choices[0].message.content)

ChatCompletion(id='chatcmpl-9bOYlHAMwsZbo6CILL26MzBKJ4R9j', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{\n  "location": "北京，北京市"\n}', role='assistant', function_call=None, tool_calls=None))], created=1718699571, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=95, total_tokens=107))
{
  "location": "北京，北京市"
}


### 2.3 强制调用

`function_call`参数的最后一个模式是强制调用函数，方法也是很简单，只要在参数里面传入这个函数的名字即可。在这一部分，我们指定`name`为`get_current_weather`，这将强制它使用`get_current_weather`函数。如果我们查看结果，我们实际上会看到返回了这个`function_call`对象，其中有`name`为`get_current_weather`的一些参数。

为了好玩，这里面传了一个于天气无关的参数，然后强制他使用`get_current_weather`函数，但我们传入的参数中绝对没有任何信息关于它应该如何调用该函数。所以在这里，他编造了北京，北京市的参数，如果我们再次运行它，它会不断地调用北京市的参数。

我们还可以尝试使用不同的模型函数来调用，不同的参数值，不同的输入消息，但是这里面要注意一个事情，首先，函数本身和描述都会计入传递给OpenAI的令牌使用限制。所以如果我们运行这个，我们可以看到返回的提示令牌是95。如果我们注释掉`functions`和`function_call`，我们可以看到提示令牌减少到15。因为OpenAI模型对令牌有限制，因此，在构造要传递给OpenAI的消息时，现在不仅需要注意消息的长度，还需要注意传递的函数的长度。

In [13]:
# 无关提示强制调用函数
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"}, # 强制调用函数get_current_weather
)
print(response)
print(response.choices[0].message.function_call.arguments)

ChatCompletion(id='chatcmpl-9bOYmRZafZH6U2HUqvKcBF7QRxTtM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "北京，北京市"\n}', name='get_current_weather'), tool_calls=None))], created=1718699572, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=95, total_tokens=107))
{
  "location": "北京，北京市"
}


## 三、函数调用以及执行函数

最后，让我们看一下如何将这些函数调用和实际执行函数调用的结果传递回语言模型。这很重要，因为通常我们希望使用语言模型确定要调用的函数，然后运行该函数，但然后将其传递回语言模型以获得最终响应。

我们会经过这样的流程，首先我们问问题得到一个带有`function call`参数的响应，然后我们将这个消息添加到我们的消息列表中。然后我们可以模拟使用语言模型提供的参数调用`get_current_weather`函数，并且保存到一个变量`observation`中。接着我们定义一个新的消息列表，表示刚刚调用函数的结果，这里面有一个重要的点就是`role`等于`function`，也就是告诉语言模型这是调用函数的响应。除此之外，还传递函数的名称`name`以及`content`变量，设为上述计算的`observation`。

如果我们然后使用这个消息列表调用语言模型，我们可以看到语言模型回答的非常好：北京的天气目前是22摄氏度，天气以晴天为主，也有多云的情况。

In [14]:
# 与天气无关提示调用
messages = [
    {
        "role": "user",
        "content": "你好!",
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)

print(response)

ChatCompletion(id='chatcmpl-9bOYnmOrdCsVgpDCeX89QN20sa8X8', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='你好!有什么可以帮助你的吗？', role='assistant', function_call=None, tool_calls=None))], created=1718699573, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=18, prompt_tokens=88, total_tokens=106))


由于编码问题，我们可以单独答应对应的返回的文本信息

In [15]:
print(response.choices[0].message.content)

你好!有什么可以帮助你的吗？


## 四、英文版提示

**1.1 简单例子：得到当前天气**

In [16]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

**1.2 定义function函数**

In [17]:
# define a function
functions = [
    {
        "name": "get_current_weather",
        "description": "Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
            },
            "required": ["location"],
        },
    }
]

**1.3 相关提示调用结果**

In [18]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston?"
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions
)

print(response)

ChatCompletion(id='chatcmpl-9bOYoD5cHnnQMC942sjMHP5GVth8x', choices=[Choice(finish_reason='function_call', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n"location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1718699574, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=17, prompt_tokens=82, total_tokens=99))


**1.4 无关提示调用结果**

In [19]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]

response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
)

**2.1 自动判断是否调用**

自动判断是否调用函数，判断提示是否和函数相关

In [20]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="auto",
)
print(response)

ChatCompletion(id='chatcmpl-9bOYqmeZ73y5pfew7QuN5nLnDdUTU', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1718699576, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=10, prompt_tokens=76, total_tokens=86))


**2.2 强制不调用**

无关提示强制不调用，结果正常

In [21]:
messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call="none",
)
print(response)

ChatCompletion(id='chatcmpl-9bOYrnwQwIQ0mMuSVPBJdSVXue22m', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Hello! How can I assist you today?', role='assistant', function_call=None, tool_calls=None))], created=1718699577, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=9, prompt_tokens=77, total_tokens=86))


**2.3 强制调用**

强制无关问题的调用

In [22]:

messages = [
    {
        "role": "user",
        "content": "hi!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9bOYsUNJuRiQNwhfQ1BiB79KqCDKq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "San Francisco, CA"\n}', name='get_current_weather'), tool_calls=None))], created=1718699578, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=12, prompt_tokens=83, total_tokens=95))


强制有关问题调用，结果和`auto`一样

In [23]:
messages = [
    {
        "role": "user",
        "content": "What's the weather like in Boston!",
    }
]
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=messages,
    functions=functions,
    function_call={"name": "get_current_weather"},
)
print(response)

ChatCompletion(id='chatcmpl-9bOYuDhDjo4lIbzeNqj9zgngJ0cVB', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content=None, role='assistant', function_call=FunctionCall(arguments='{\n  "location": "Boston, MA"\n}', name='get_current_weather'), tool_calls=None))], created=1718699580, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=11, prompt_tokens=89, total_tokens=100))
